# * Sales Channel

In [1]:
import configparser
import datetime as dt
import pandas as pd
import numpy as np
import xlrd
import oracledb
import re

config = configparser.ConfigParser()
config.read('../../my_config.ini')
config.sections()

TDMDBPR_user = config['TDMDBPR']['username']
TDMDBPR_pwd = config['TDMDBPR']['password']
TDMDBPR_db = config['TDMDBPR']['db']
TDMDBPR_host = config['TDMDBPR']['host']
TDMDBPR_port = config['TDMDBPR']['port']

AKPIPRD_user = config['AKPIPRD']['username']
AKPIPRD_pwd = config['AKPIPRD']['password']
AKPIPRD_db = config['AKPIPRD']['db']
AKPIPRD_host = config['AKPIPRD']['host']
AKPIPRD_port = config['AKPIPRD']['port']

curr_dt = dt.datetime.now().date()
str_curr_dt = curr_dt.strftime('%Y%m%d')

In [2]:
''' Input parameter '''

op_dir = 'data'
op_sales_file = f'sales_channel_{str_curr_dt}'

# v_year = 0
# v_month_start = 0
# v_month_end = 0
# v_date = 20250101

print(f'\nParameter input...\n')
print(f'   -> op_dir: {op_dir}')
print(f'   -> op_sales_file: {op_sales_file}')
# print(f'\n   -> v_year: {v_year}')
# print(f'   -> v_month_start: {v_month_start}')
# print(f'   -> v_month_end: {v_month_end}')
# print(f'\n   -> v_date: {v_date}')


Parameter input...

   -> op_dir: data
   -> op_sales_file: sales_channel_20251127


## Import Transaction
-> DASH_PERFORMANCE

In [3]:
''' Execute transaction '''


# Input parameter
v_start_date = 20250101
print(f'\nParameter input...')
print(f'   -> v_start_date: {v_start_date}')

curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
print(f'\nData as of {curr_datetime}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
src_cur = src_conn.cursor()

query = (f"""
    SELECT /*+PARALLEL(8)*/ 
        TM_KEY_MTH, COMPANY, PRODUCT, SUB_PRODUCT, GP_SKU, SPECIAL_SKU, PROJECT_GROUP, CUST_TYPE, BU_CHANNEL, TDS_CHANNEL, TDS_SUB_CHANNEL, GROUP_TDS_SPECIAL, TDS_SPECIAL_CHANNEL, CHANNEL_GROUP, BUNDLE_TYPE
        , ORGID_H, HOP_HINT, D_CLUSTER, ORGID_G, TDS_GMD, TDS_SGMD, DISTRICT_TH, DISTRICT_EN
        , A.CCAA, O.PROVINCE_CD, O.PROVINCE_ENG, O.PROVINCE_TH
        , TARGET, ACTUAL, RUNRATE, TARGET_VALUE, ACTIVATION_VALUE, ACTIVATION_VALUE_RUNRATE
    --	, CASE  WHEN SUB_PRODUCT = 'POSTPAY' AND CHANNEL_GROUP <> 'B2B' THEN 'INDIVIDUAL' 
    --			WHEN SUB_PRODUCT = 'PREPAY' 
    --				THEN (CASE 	WHEN GP_SKU IN ('MASS', '7-11 TRUE-7', 'STUDENT', 'TRUE ID SIM', 'TRUE LOTUS X2') THEN 'MASS' 
    --							WHEN GP_SKU IN ('MIGRANT', 'AEC') THEN 'MIGRANT' 
    --							WHEN GP_SKU IN ('TOURIST', 'TOURIST-INTER') THEN 'TOURIST'
    --							END) 
    --			WHEN SUB_PRODUCT = 'TOL' AND GP_SKU = 'Consumer Service' THEN 'CONSUMER' 
    --			WHEN SUB_PRODUCT = 'TVS NOW APP' THEN 'TVS NOW'
    --			END SEGMENT
        
    FROM (
        SELECT SUBSTR(TM_KEY_DAY, 1, 6) TM_KEY_MTH
            , COMPANY, PRODUCT, SUB_PRODUCT, GP_SKU, SPECIAL_SKU, PROJECT_GROUP, CUST_TYPE, BU_CHANNEL, TDS_CHANNEL, TDS_SUB_CHANNEL, GROUP_TDS_SPECIAL, TDS_SPECIAL_CHANNEL, CHANNEL_GROUP, BUNDLE_TYPE 
            , ORGID_H, HOP_HINT, D_CLUSTER, ORGID_G, TDS_GMD, TDS_SGMD, DISTRICT_TH, DISTRICT_EN
            , SUBSTR(PARTNER_CCAATT, 1, 4) AS CCAA --, PARTNER_CCAATT, PARTNER_SCAB
            , SUM(TARGET) TARGET, SUM(ACTUAL) ACTUAL, SUM(RUNRATE) RUNRATE, SUM(TARGET_VALUE) TARGET_VALUE, SUM(ACTIVATION_VALUE) ACTIVATION_VALUE, SUM(ACTIVATION_VALUE_RUNRATE) ACTIVATION_VALUE_RUNRATE
            
        FROM CDSSHRE.DASH_PERFORMANCE
        WHERE TM_KEY_DAY >= {v_start_date}
        AND ( (SUB_PRODUCT = 'POSTPAY' AND CUST_TYPE = 'INDY')
            OR (SUB_PRODUCT = 'PREPAY' AND CUST_TYPE = 'INDY')
            OR (SUB_PRODUCT = 'TOL') --AND GP_SKU = 'Consumer Service'
            OR SUB_PRODUCT IN ('TVS NOW APP', 'CMDU') )

        GROUP BY SUBSTR(TM_KEY_DAY, 1, 6) 
            , COMPANY, PRODUCT, SUB_PRODUCT, GP_SKU, SPECIAL_SKU, PROJECT_GROUP, CUST_TYPE, BU_CHANNEL, TDS_CHANNEL, TDS_SUB_CHANNEL, GROUP_TDS_SPECIAL, TDS_SPECIAL_CHANNEL, CHANNEL_GROUP, BUNDLE_TYPE 
            , ORGID_H, HOP_HINT, D_CLUSTER, ORGID_G, TDS_GMD, TDS_SGMD, DISTRICT_TH, DISTRICT_EN
            , SUBSTR(PARTNER_CCAATT, 1, 4)
    ) A

    LEFT JOIN (
        SELECT DISTINCT SUBSTR(CCAATT,1,4) AS CCAA--, DISTRICT_EN
            , SUBSTR(CCAATT,1,2) AS PROVINCE_CD, PROVINCE_ENG, PROVINCE_TH
        FROM CDSAPPO.DIM_MOOC_AREA
        WHERE TEAM_CODE <> 'ไม่ระบุ' AND REMARK <> 'Dummy'
    ) O
        ON O.CCAA = A.CCAA
""")


try:
    # Create Dataframe
    src_cur.execute(query)
    rows = src_cur.fetchall()
    src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])
    print(f'\nDataFrame: {src_df.shape[0]} rows, {src_df.shape[1]} columns')

    # Generate CSV file
    src_df.to_csv(f'{op_dir}/{op_sales_file}.csv', index=False, encoding='utf-8')
    print(f'\n   -> Generate "{op_sales_file}.csv" successfully')

    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()


Parameter input...
   -> v_start_date: 20250101

Data as of 2025-11-27, 13:30:29

DataFrame: 595401 rows, 34 columns

   -> Generate "sales_channel_20251127.csv" successfully


## Review

In [5]:
src_df

,TM_KEY_MTH,COMPANY,PRODUCT,SUB_PRODUCT,GP_SKU,SPECIAL_SKU,PROJECT_GROUP,CUST_TYPE,BU_CHANNEL,TDS_CHANNEL,...,PROVINCE_CD,PROVINCE_ENG,PROVINCE_TH,TARGET,ACTUAL,RUNRATE,TARGET_VALUE,ACTIVATION_VALUE,ACTIVATION_VALUE_RUNRATE,SEGMENT
0,202502,DTAC,PREPAY,PREPAY,MASS,MASS,None,INDY,DTAC Channel,Unidentified,...,10,Bangkok,กรุงเทพมหานคร,0.00,482,482.0,0.00,0.0,0.0,MASS
1,202503,TRUE,PREPAY,PREPAY,MASS,MASS,MASS,INDY,7-11,7-11,...,40,Khon Kaen,ขอนแก่น,0.00,1638,1638.0,0.00,0.0,0.0,MASS
2,202503,TRUE,PREPAY,PREPAY,MASS,MASS,MASS,INDY,Non-Shop,Wholesales,...,60,Nakhon Sawan,นครสวรรค์,0.00,78,78.0,0.00,0.0,0.0,MASS
3,202509,TRUE,PREPAY,PREPAY,MIGRANT,MASS,MASS,INDY,Non-Shop,Wholesales,...,26,Nakhon Nayok,นครนายก,0.00,11,11.0,0.00,0.0,0.0,MIGRANT
4,202510,TRUE,PREPAY,PREPAY,MASS,MASS,MASS,INDY,7-11,7-11,...,10,Bangkok,กรุงเทพมหานคร,0.00,489,489.0,0.00,0.0,0.0,MASS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595396,202504,DTAC,PREPAY,PREPAY,MASS,MASS,None,INDY,DTAC Channel,Unidentified,...,10,Bangkok,กรุงเทพมหานคร,0.00,101,101.0,0.00,0.0,0.0,MASS
595397,202504,TRUE,TVS,CMDU,CMDU,CMDU,CMDU,INDY,Retail Shop,True Shop WW,...,10,Bangkok,กรุงเทพมหานคร,0.48,0,0.0,3223.79,0.0,0.0,None
595398,202504,DTAC,PREPAY,PREPAY,MASS,MASS,None,INDY,DTAC Channel,Unidentified,...,20,Chon Buri,ชลบุรี,0.00,103,103.0,0.00,0.0,0.0,MASS
595399,202504,DTAC,PREPAY,PREPAY,MASS_MIGRANT,MASS_MIGRANT,None,INDY,DTAC Channel,Unidentified,...,39,Nong Bua Lam Phu,หนองบัวลำภู,260.00,0,0.0,0.00,0.0,0.0,None
